In [31]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [32]:
if "train_x" not in globals():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))



In [44]:
lib.utils.set_seed()
# network = lib.nets.PYRAMID()
#network = lib.nets.create_livenet_odd_2()
# train_x, train_y = lib.datasets.get_odd()
# network = lib.livenet.LiveNet(train_x.shape[1], None, torch.max(train_y) + 1)
network = lib.nets.PERCEPTRON(train_x.shape[1], torch.max(train_y) + 1, l1=0.01)
criterion = lib.nets.criterion_n

def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.livenet.LiveNetOptimizer(net, decay=0.0)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        optimizer = torch.optim.Adam(net.parameters(), betas=(0.0, 0.95))
        # optimizer = torch.optim.Adam(net.parameters())
    return optimizer

optimizer = create_optimizer(network)


Torch


In [45]:
params = [p for p in network.named_parameters()]
logits = network.forward(train_x)
for name, p in params:
    print(f"{name} {p.detach().numpy()}")
pred = network.forward(train_x)
loss = criterion(pred, train_y)
print(loss)
pred.detach().numpy()
# torch.onnx.export(network, train_x, "/home/spometun/model.onnx", verbose=False)




linear1.weight [[-0.     0.019 -0.029 ...  0.022  0.004  0.002]
 [-0.02  -0.015 -0.01  ... -0.02  -0.006 -0.03 ]]
linear1.bias [-0.02   0.015]
tensor(1.0939, grad_fn=<DivBackward0>)


array([[ 0.273,  0.635],
       [-0.329,  0.542],
       [ 0.153, -0.004],
       ...,
       [ 0.304,  0.578],
       [ 0.077,  0.333],
       [-0.019,  0.095]], dtype=float32)

In [51]:
class Trainer:
    def __init__(self, network: torch.nn.Module, batch_iterator: typing.Iterator,
                  criterion: typing.Callable, optimizer, epoch_size=1):
        self.network = network
        self.batch_iterator = batch_iterator
        self.criterion = criterion
        self.optimizer = optimizer
        self.epoch_size = epoch_size
        self.history = []
        self.counter = 0

    def step(self, n_steps=1):
        for _ in range(n_steps):
            self._step()

    def _step(self):
        data, labels = next(self.batch_iterator)
        pred = self.network.forward(data)
        loss = self.criterion(pred, labels)
        reg_loss = self.network.internal_loss()
        all_loss = loss + reg_loss
        self.optimizer.zero_grad()
        all_loss.backward()
        if self.counter % self.epoch_size == 0:
            params = lib.utils.get_parameters_dict(self.network)
            grads = lib.utils.get_gradients_dict(self.network)
            self.history.append({"params": params, "grads": grads})
            LOG(f"{all_loss.detach().numpy():.3f} = {loss.detach().numpy():.3f}+{reg_loss.detach().numpy():.3f}")
        self.optimizer.step()
        self.counter += 1




In [62]:
lib.utils.set_seed()
network = lib.nets.PERCEPTRON(train_x.shape[1], torch.max(train_y) + 1, l1=0.01)
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = lib.nets.criterion_n
optimizer = create_optimizer(network)
trainer = Trainer(network, batch_iterator, criterion, optimizer, epoch_size=10)


Torch


In [64]:

trainer.step(100)


Iˈ0.000 0.689 = 0.479+0.209 
Iˈ0.009 0.731 = 0.526+0.204 
Iˈ0.049 0.688 = 0.493+0.195 
Iˈ0.057 0.686 = 0.501+0.184 
Iˈ0.065 0.663 = 0.482+0.181 
Iˈ0.073 0.618 = 0.444+0.174 
Iˈ0.080 0.684 = 0.513+0.171 
Iˈ0.087 0.623 = 0.460+0.163 
Iˈ0.129 0.657 = 0.497+0.160 
Iˈ0.137 0.619 = 0.466+0.154 


In [48]:
params_history = []
optimizer.param_groups[0]["lr"]



0.0010

In [108]:
optimizer.param_groups[0]["lr"] = 0.001
network.alpha_l1 = 0.01
batch_size = 1000

n_epochs = 8
for i in range(n_epochs):
    e_counter = 0
    for start, end in lib.gen_utils.index_batcher(batch_size, len(train_x)):
        x = train_x[start:end]
        y = train_y[start:end]
        pred = network.forward(x)
        loss = criterion(pred, y)
        reg_loss = network.internal_loss()
        all_loss = loss + reg_loss
        optimizer.zero_grad()
        all_loss.backward()

        params = lib.utils.get_parameters_dict(network)
        grads = lib.utils.get_gradients_dict(network)
        params_history.append({"params": params, "grads": grads})

        optimizer.step()
        if e_counter == 0:
            LOG(f"{all_loss.detach().numpy():.3f} = {loss.detach().numpy():.3f}+{reg_loss.detach().numpy():.3f}")
        e_counter += 1



Iˈ0.000 1.391 = 1.105+0.286 
Iˈ0.048 0.761 = 0.520+0.241 
Iˈ0.093 0.704 = 0.508+0.196 
Iˈ0.138 0.626 = 0.467+0.159 
Iˈ0.179 0.603 = 0.467+0.135 
Iˈ0.223 0.583 = 0.463+0.121 
Iˈ0.275 0.569 = 0.454+0.115 
Iˈ0.323 0.580 = 0.461+0.119 


In [137]:

params_orig = [p.detach().clone().numpy() for p in network.parameters()]


In [138]:
# params = copy.deepcopy(params_orig)
#lib.utils.add_noise_to_params(params, 1, .0)
# lib.utils.set_parameters(network, params)


In [141]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.1886

In [140]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    val0 = param["params"]["linear1.weight"][0][1].item()
    return val0
    return param["grads"]["linear1.weight"][0][0].item() / 1000
    # return np.max(np.abs(param[0].numpy()))

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(params_history, param_picker0)
# plt.plot(values)
values = get_param_values(params_history, param_picker1)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(network.parameters())
#accum.add_value(params_history[-1])
accum.plot()




Using matplotlib backend: TkAgg


In [39]:
def form_balanced(x, y, n):
    assert n % 2 == 0
    n //= 2
    y = y.numpy()
    x = x.numpy()
    inds = np.argsort(y.squeeze(1), axis=0)
    y = y[inds]
    x = x[inds]
    i = 0
    batches_x = []
    batches_y = []
    while True:
        a = y[i:i + n]
        b = y[len(y) - i - n: len(y) - i]
        if not (a==0).all() or not (b==1).all():
            break
        batches_x += [x[i:i+n], x[len(y) - i - n: len(y) - i]]
        batches_y += [a, b]
        i += n

    x = np.vstack(batches_x)
    y = np.vstack(batches_y)
    return torch.tensor(x), torch.tensor(y)

train_x, train_y = form_balanced(train_x, train_y, 1000)


In [ ]:
# list(network.parameters())[0].data = torch.Tensor([[10.], [-10]])
# list(network.parameters())[1].data = torch.Tensor([-5., 15])
np.set_printoptions(precision=2)
params = list(network.parameters())
par = []
for p in params:
    par.append(p.detach())
    print (par[-1])

In [ ]:
o1 = train_x @ par[1].T + par[2]
o1

In [ ]:
a1 = torch.sigmoid(o1)
a1

In [ ]:
o2 = a1[:, 0] + a1[:, 1] - 1
mseloss(o2, train_y)

In [ ]:
o2 @ par[2].T

In [ ]:
network.forward(train_x)

In [ ]:
1345639 * 5145

In [ ]:
import pickle
import numpy as np

class A:
    def __init__(self):
        self.v = 1
    def __del__(self):
        print(f"del {self.v}")

a = A()
b = A()
b.v = 2
l = [b]
print("start")
del a
print("after del")
l[0].v
a.v
